#  Simple Stellar Population Synthesis (SPS)

This is a guided activity that will also serve as the basis for Problem Set 5.  

You can work together with a partner in class but each person should submit their own answers to the questions.  If you choose to work together with someone, please indicate this on the document you turn in.


------------------------------

## Introduction
To model the spectral energy distribution of a simple stellar population we need three things:

1.   Models of the spectral energy distributions of stars as a function of temperature, gravity, and composition - stellar atmospheres
2.   Models of the properties of stars as a function of stellar mass and time - stellar evolution
3.   A characterization of the number of stars as a function of stellar mass - initial mass function

In this notebook we'll cover all three in turn, make some simple stellar population models at the end, and see how they evolve as a function of redshift.

To begin with, we need to load some modules.

In [ ]:
from pylab import *
from scipy.integrate import simps

import warnings  #these lines get rid of some annoying errors . . .
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Constants

This section sets the values of some constants.  All of these are in cgs units.  These include Planck's constant $h$, Boltzmann's constant $k_B$, the speed of light $c$, and the Stefan-Boltzmann constant $\sigma$, which is derived from the other three.  This is followed by some solar units, $M_{\odot}$, $R_{\odot}$, $L_{\odot}$, and $T_{\rm eff,\odot}$.

In [ ]:
#Physical
h=6.6261e-27
kB=1.3807e-16
c=2.99792458e10
sigma=2*pow(pi,5)*pow(kB,4)/(15*c*c*h*h*h)

#Astronomical
Msun = 1.99e33 #g
Rsun = 6.957e10 #cm
Lsun = 3.828e33 #erg/s
Tsun = 5772. #K
AU = 1.496e13 #cm
pc = 206265*AU #cm
pc10 = 10*pc #10 parsecs [in cm]
const = 4*pi*pc10*pc10 #factor for absolute magnitude

## Spectral Energy Distribution - The Planck Function

The Planck function describes the energy emitted by a blackbody as a function of temperature and wavelength.

In [ ]:
def B(w,T):
  term1 = 2*h*c*c*pow(w,-5)
  term2 = (h*c)/(w*kB*T)
  term3 = 1.0/(exp(term2) - 1.0)
  return term1*term3

### Plotting the Planck Function

A blackbody emits thermal radiation but it has no notion of chemistry, emission or absorption lines.  The hotter it is, the more radiation it should emit.  The peak should also shift to shorter wavelength as temperature increases.  This phenomenon is encapsulated by Wien's law: $\lambda_{\rm peak} [{\rm A}] = 2.898\times10^7/T [K]$.  Let's see what that looks like.

In [ ]:
### FIGURE 1
fs=18
figure(1,figsize=(10,10))
wav=logspace(1,6,10000) #in Angstrom
wav_cm = wav*1e-8 #convert Angstrom to cm
semilogy(wav,B(wav_cm,3000),label="3,000K",color='Red')
semilogy(wav,B(wav_cm,5000),label='5,000K',color='Green')
semilogy(wav,B(wav_cm,7000),label='7,000K',color='Blue')
axvline(2.898e7/3000, ls=":",color='Red')
axvline(2.898e7/5000, ls=":",color='Green')
axvline(2.898e7/7000, ls=":",color='Blue')
xlim(800,20000)
ylim(1e9,1e15)
xticks(fontsize=fs)
yticks(fontsize=fs)
xlabel(r'$\lambda~(\AA)$',fontsize=fs)
ylabel(r'$B_{\lambda}(\lambda,T)$',fontsize=fs)
legend(loc='lower right',fontsize=fs);

### Sanity Check
Does it work?  If we have implemented things correctly, then the integral of $B_{\lambda}(\lambda,T)$ over all wavelengths will be equal to $\frac{\sigma}{\pi} T^4$.  Here we check the ratio, it should be 1.   

In [ ]:
numerator = simps(x=wav_cm,y=B(wav_cm,5000)) #the integral over wavelength
denominator = (sigma/pi)*pow(5000,4) #from the Stefan-Boltzmann law

ratio = numerator/denominator

print("{0:6.4f}".format(ratio))

To get the (BB) spectral energy density of a star with a given lumonsity, we can use the Planck function and the Stefan-Boltzmann law: $L_{SB}=4 \pi R^2 \sigma T^4$.

In [ ]:
def SED(w,R,T):
  return 4*pow(pi*R,2)*B(w,T)

def L_SB(R,T):
  return 4*pi*pow(R,2)*sigma*pow(T,4)

Now we can check this by seeing if the frequency integral over flux for $1~R_{\odot}$ and $T=5,778\,K$ gives $1~L_{\odot}$.

In [ ]:
R=Rsun
T=Tsun
seds=SED(wav_cm,R,T)
L = simps(x=wav_cm,y=seds)
ratio = L/Lsun
ratio2 = L/L_SB(R,T)
print("{0:6.4f} {1:6.4f}".format(ratio, ratio2))

And plot the SED for this case.

In [ ]:
### FIGURE 2
fs=18
figure(2,figsize=(10,10))
plot(wav,seds)
xlim(1000,20000)
xticks(fontsize=fs)
yticks(fontsize=fs)
xlabel(r'$\lambda (\AA)$',fontsize=fs)
ylabel(r'Specral Energy Density (erg/s/cm)',fontsize=fs);

Modern SPS models are obviously not made using BB spectra.  However, we can gain some considerable insight into SPS models with this simple approach.

## Isochrones

Isochrones are a data product of stellar evolution codes: they give a picture of what a stellar population should look like in terms of basic, global parameters (temperature, radius, luminosity, etc.) as a function of initial stellar mass and the age of the population.  In the cell below, download an isochrone data file that I've prepared for this exercise.

The cell below contains some code for reading in and storing the file above.  Just execute the cell.

In [ ]:
#@title
class ISO:
    def __init__(self, filename, verbose=True):
        self.filename = filename
        if verbose:
            print('Reading in: ' + self.filename)

        self.version, self.abun, self.rot, self.ages, self.num_ages, self.hdr_list, self.isos = self.read_iso_file()

    def read_iso_file(self):
        #open file and read it in
        with open(self.filename) as f:
            content = [line.split() for line in f]
        version = {'MIST': content[0][-1], 'MESA': content[1][-1]}
        print (version)
        abun = {content[3][i]:float(content[4][i]) for i in range(1,5)}
        rot = float(content[4][-1])
        num_ages = int(content[6][-1])

        #read one block for each isochrone
        iso_set = []
        ages = []
        counter = 0
        data = content[8:]
        for i_age in range(num_ages):
            #grab info for each isochrone
            num_eeps = int(data[counter][-2])
            num_cols = int(data[counter][-1])
            hdr_list = data[counter+2][1:]
            formats = tuple([np.int32]+[np.float64 for i in range(num_cols-1)])
            iso = np.zeros((num_eeps),{'names':tuple(hdr_list),'formats':tuple(formats)})
            #read through EEPs for each isochrone
            for eep in range(num_eeps):
                iso_chunk = data[3+counter+eep]
                iso[eep]=tuple(iso_chunk)
            iso_set.append(iso)
            ages.append(iso[0][1])
            counter+= 3+num_eeps+2
        return version, abun, rot, ages, num_ages, hdr_list, iso_set

    def age_index(self, age):
        diff_arr = abs(np.array(self.ages) - age)
        age_index = np.where(diff_arr == min(diff_arr))[0][0]

        if ((age > max(self.ages)) | (age < min(self.ages))):
            print('The requested age is outside the range. Try between ' + str(min(self.ages)) + ' and ' + str(max(self.ages)))

        return age_index


Now we're going to load the isochrones into memory so that we can use them.

In [ ]:
MIST=ISO('./data/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.4_basic.iso')

This file contains isochrones for log10(age [yr]) between 5 and 10.3.  In other words, between 100,000 and about 20 billion years (= 20 Gyr).  We can access different ages by index.  If you want to find the index that corresponds to log10(age) = 8 ($10^8$ years), you can use the `age_index` function.  

In [ ]:
MIST.age_index(8)

Below we'll make the Hertzsprung-Russell Diagram (HRD) for an isochrone with an age of 10 Gyr.  The HRD plots the effective temperature ($T_{eff}$) along the horizontal axis and the luminosity ($L$) along the vertical.  Convention is to plot the luminosity in solar units ($L_{\odot}$) but here we just use cgs units.  Convention also dictates that we plot $T_{eff}$ increasing to the left.  This is consistent with the way that we will plot the color-magnitude diagram later.

In [ ]:
### FIGURE 3
figure(3,figsize=(10,10))
xpts=[]
ypts=[]
zpts=[]
iso=MIST.isos[100]
for i in iso:
  eep=i['EEP']
  mass=i['initial_mass']
  if 0.5 <= mass <= 100 and eep < 808:
    Teff = pow(10,i['log_Teff'])
    R = pow(10,i['log_R'])*Rsun
    Lum = pow(10,i['log_L'])*Lsun
    Lsb = L_SB(R,Teff)
    xpts.append(Teff)
    ypts.append(Lsb)
    zpts.append(Lum)
fs=18
semilogy(xpts,ypts,color='DodgerBlue',label='Lum from SB')
semilogy(xpts,zpts,color='Tomato',label='Lum from model',ls='--')
legend(fontsize=fs)
xlim(6000,2750)
xticks(fontsize=fs)
yticks(fontsize=fs)
xlabel(r"${\rm T_{eff}~[K]}$",fontsize=fs)
ylabel(r"${\rm Luminosity~[erg/s]}$",fontsize=fs);

The isochrone ploted above does not show the full range of initial mass or evolutionary phase.  The two ways of obtaining the luminosity should be identical, and they appear to be.

## Initial Mass Function

For our purposes we'll only work with power-law IMFs.  This is not a major limitation since most widely-used IMFs behave this way for masses greater than about $0.5 M_{\odot}$.  We'll also restrict ourselves to maximum range of $0.5 \leq M/M_{\odot} \leq 100$ in all cases.  Stars with initial masses greater than $100 M_{\odot}$ are extremely rare and difficult to quantify.  The powerlaw IMF is usually denoted $\xi(M) = \frac{dN}{dM} \propto M^{\alpha}$.  The most commonly used value is $\alpha=-2.35$, this is called the Salpeter IMF.

The power-law IMF can be integrated to give an analytical expression for the total number of stars ($N_{tot}$) between two masses.  

$N_{tot} = \int_{M_1}^{M_2} \xi(M)\,dM$

Be careful in the case of $\alpha=-1$. (Why?)

We can normalize these functions by setting the total mass.  For this we can choose $1M_{\odot}$ so that later, if we want to scale a galaxy up to, say, $10^8 M_{\odot}$, we can simply multiply it by the total mass we want in $M_{\odot}$.  Note that there is a subtlety to normalizing in this way.  We'll get to that when we assemble a full population model in the next section.

In [ ]:
def Ntot(M1,M2,alpha):
  a1=alpha+1.
  a2=alpha+2.
  norm = 1/(pow(100,a2)-pow(0.5,a2))
  return norm*(a2/a1)*(pow(M2,a1)-pow(M1,a1))

In the code cell below, run the Ntot function for M1=0.5, M2=100, alpha=-2.35 and -1.01.

In [ ]:
Ntot(0.5,100.,-2.35)

In [ ]:
Ntot(0.5,100,-4)

Likewise the total mass ($M_{tot}$) can be obtained from:

$M_{tot} = \int_{M_1}^{M_2} M\,\xi(M)\,dM$

In [ ]:
def Mtot(M1,M2,alpha):
  a2=alpha+2.0
  norm=1/(pow(100,a2)-pow(0.5,a2))
  return norm*(pow(M2,a2)-pow(M1,a2))

In the cell below, evaluate the Mtot function for M1=0.5, M2=100, and alpha=-2.35 and -1.01.  We should get 1.0 ($M_{\odot}$ units) by design.

In [ ]:
Mtot(0.5,100,-2.35)

In [ ]:
Mtot(0.5,100,-1.01)

Here is a plot that shows the effect of changing the IMF $\alpha$ value.  We try values of -1.01, -1.5, -2.35, and -4. Then we plot the *Cumulative Number Fraction* as a function of stellar mass over the range.  Note also that the x-axis is plotted logarithmically.

In [ ]:
### FIGURE 4
figure(4,figsize=(10,10))
masses=logspace(-0.30103,2,50)
CNF1=[]
CNF2=[]
CNF3=[]
CNF4=[]
for m in masses:
  CNF1.append( Ntot(0.5,m,-1.01)/Ntot(masses[0],masses[-1],-1.01))
  CNF2.append( Ntot(0.5,m,-1.5)/Ntot(masses[0],masses[-1],-1.5))
  CNF3.append( Ntot(0.5,m,-2.35)/Ntot(masses[0],masses[-1],-2.35))
  CNF4.append( Ntot(0.5,m,-4)/Ntot(masses[0],masses[-1],-4))
semilogx(masses,CNF1,color='DodgerBlue',label='-1.01')
semilogx(masses,CNF2,color='Tomato',label='-1.5')
semilogx(masses,CNF3,color='DarkGreen',label='-2.35')
semilogx(masses,CNF4,color='Magenta',label='-4')
axhline(0.5,ls='--')
axvline(1.0,ls='--')
xticks(fontsize=fs)
yticks(fontsize=fs)
legend(fontsize=fs,title=r'$\alpha$', title_fontsize=fs)
ylabel('Cumulative Number Fraction',fontsize=fs)
xlabel(r'Mass $(M_{\odot})$',fontsize=fs);

What do you notice that is different about the range of curves shown above?  An IMF is called "bottom-heavy" if the majority of the stars have lower masses and "top-heavy" if the majority have higher masses.  If the dividing line between "bottom-heavy" and "top-heavy" is drawn at $1~M_{\odot}$, how would you characterize IMFs defined by the above values of $\alpha$?

## Putting it all together

Now that we've assembled each of the three pieces, we are in a place to create the "integrated spectrum" of a stellar population.  This requires that we:

*   Choose an isochrone (age of the population)
*   Choose an IMF $\alpha$

Once we've made that choice, we can loop over the points in the isochrone, calculate how many stars are present based on their masses.  Then we create a BB spectrum for each of these "bins" of stars and add them together.


In [ ]:
def SPS(iso, alpha=-2.35, Mmin=0.5, Mmax=100., z=0.):
  wav=logspace(1,6,10000) #in Angstrom
  wav_cm = wav*1e-8 #convert Angstrom to cm
  ISED = zeros(shape(wav))
  Msum=0.0
  Nsum=0.0
  for i in range(len(iso)-1):
    M1=iso[i]['initial_mass']
    M2=iso[i+1]['initial_mass']
    if M1 > Mmin and M2 < Mmax:
      logR1 = iso[i]['log_R']
      logR2 = iso[i+1]['log_R']
      logT1 = iso[i]['log_Teff']
      logT2 = iso[i+1]['log_Teff']

      R = 0.5*Rsun*(pow(10,logR1) + pow(10,logR2))
      T = 0.5*(pow(10,logT1)+pow(10,logT2))
      N = Ntot(M1,M2,alpha)
      M = Mtot(M1,M2,alpha)
      ISED += N*SED(wav_cm,R,T)
      Nsum += N
      Msum += M
  wav = wav*(1+z)
  return Msum, Nsum, wav, ISED

Here we plot the integrated SED of a 10 Gyr population (isochrone index 100) for two different IMF alpha's.

In [ ]:
### FIGURE 5
figure(5,figsize=(10,10))

index=MIST.age_index(8)
iso=MIST.isos[index]

alpha=-2.35
M,N,w,S=SPS(iso, alpha)
print('Total mass, number = {0:5.3f}, {1:5.3f} for alpha = {2:4.2f}'.format(M,N,alpha))
plot(w,S,color='DodgerBlue',label='integrated alpha=-2.35')

alpha= -1.35
M,N,w,S=SPS(iso,alpha)
print('Total mass, number = {0:5.3f}, {1:5.3f} for alpha = {2:4.2f}'.format(M,N,alpha))
plot(w,S,color='DarkGreen',label='integrated alpha=-1.35')

xlim(1000,20000)
fs=18
legend(fontsize=fs)
xticks(fontsize=fs)
yticks(fontsize=fs)
xlabel(r'$\lambda (\AA)$',fontsize=fs)
ylabel(r'Specral Energy Density (erg/s/$\AA$)',fontsize=fs);

Try running the plot above again, but this time use an age of $10^8$ years.

Now we come to the subtlety mentioned before: the way that we normalized our IMF is by setting the total mass between 0.5 and 100 $M_{\odot}$ to 1 $M_{\odot}$.  But the largest initial mass in the isochrone that we used in the first example, which has an age of 10 Gyr, is:

In [ ]:
max(MIST.isos[100]['initial_mass'][0:808])

So just above 1 $M_{\odot}$.  That means masses from 1 to 100 $M_{\odot}$ are no longer "alive" in this model but our normalization includes them.

Now lets make a similar plot but vary the log10(age)=7, 8, 9, 10.  In this we'll actually plot two things side-by-side.

In [ ]:
### FIGURE 6
figure(6,figsize=(20,10))
subplot(121)

ages=[7,8,9,10]

#the first section of this figure plots the isochrones in the HRD
for age in ages:
  index=MIST.age_index(age)
  iso=MIST.isos[index]
  plot(iso['log_Teff'], iso['log_L'])

title('Isochrones',fontsize=fs)
xticks(fontsize=fs)
yticks(fontsize=fs)
xlim(6,3)
ylim(-1.5,5.5)
xlabel(r'$\log(T_{\rm eff}~[K])$',fontsize=fs)
ylabel(r'$\log(L/L_{\odot})$',fontsize=fs);

#the second section plots the SEDs for Salpeter IMF
alpha=-2.35


subplot(122)
for age in ages:
  index=MIST.age_index(age)
  iso=MIST.isos[index]
  M,N,w,S=SPS(iso, alpha)
  loglog(w,S)
  print("Log(age)={0:5.2f} and Mass={1:5.2f}".format(age,M))

title("SEDs",fontsize=fs)
xlim(100,100000)
ylim(1e34,1e41)
fs=18
xticks(fontsize=fs)
yticks(fontsize=fs)
xlabel(r'$\lambda (\AA)$',fontsize=fs)
ylabel(r'Specral Energy Density (erg/s/cm)',fontsize=fs);

In [ ]:
w[argmax(S)]

## Magnitudes

We're now in a position to look at the evolution of our SSPs in color and magnitude space.  For this we'll use the Sloan Digital Sky Survey (SDSS) filters u,g,r,i,z.  And, actually, we'll approximate them with gaussian functions for simplicity, but these are fairly accurate representations.  The dotted line is the SED of the 10-Gyr SSP model from the previous figure.  Notice that it is brightest somewhere around g.  The dashed line is the same SED but shifted to redshift z=0.5.

In [ ]:
#these functions approximate the filters
def f(x,mu,sig): return exp(-pow(x - mu, 2.) / (2 * pow(sig, 2.)))
def thru_u(x): return f(x,3600,150)
def thru_g(x): return f(x,4750,500)
def thru_r(x): return f(x,6200,500)
def thru_i(x): return f(x,7500,500)
def thru_z(x): return f(x,8800,500)

In [ ]:
### FIGURE 7
figure(7,figsize=(20,10))
plot(wav,0.1*thru_u(wav),label='u')
plot(wav,0.35*thru_g(wav),label='g')
plot(wav,0.5*thru_r(wav),label='r')
plot(wav,0.35*thru_i(wav),label='i')
plot(wav,0.1*thru_z(wav),label='z')

#plot the SED
plot(w,0.5*S/max(S),color='k',ls=':')
plot(1.5*w,0.5*S/max(S),color='k',ls='--')

text(6500,0.5,'z=0',fontsize=fs)
text(10000,0.5,'z=0.5',fontsize=fs)
legend(fontsize=fs)
xticks(fontsize=fs)
yticks(fontsize=fs)
xlim(2000,20000)
ylim(0.001,0.55)
xlabel("Wavelength (Angstroms)",fontsize=fs)
ylabel("Throughput",fontsize=fs);

The cell below contains a function that calculates the u, g, r, i, and z absolute magnitudes for an input isochrone, IMF alpha, and redshift z.  It uses the definition of the AB magnitude system.

$m = -2.5 \log_{10} \left( \frac{\int SED(\lambda) T(\lambda) \lambda d\lambda}{\int T(\lambda) \lambda d\lambda} \right) - 48.60$

Where T is the "throughput", or the amount of light transmitted through the filter as a function of wavelength and the SED comes from our model above.  The throughput for each filter (approximately) is displayed above in Figure 7.

In [ ]:
def ugriz_mags(iso, alpha, z=0.0):
  M,N,w,S=SPS(iso=iso,alpha=alpha,z=z)
  wav_cm = 1e-8*w
  u=-2.5*log10(simps(x=wav_cm, y=wav_cm*S*thru_u(w))/(const*simps(x=w,y=w*thru_u(w)))) - 48.60
  g=-2.5*log10(simps(x=wav_cm, y=wav_cm*S*thru_g(w))/(const*simps(x=w,y=w*thru_g(w)))) - 48.60
  r=-2.5*log10(simps(x=wav_cm, y=wav_cm*S*thru_r(w))/(const*simps(x=w,y=w*thru_r(w)))) - 48.60
  i=-2.5*log10(simps(x=wav_cm, y=wav_cm*S*thru_i(w))/(const*simps(x=w,y=w*thru_i(w)))) - 48.60
  z=-2.5*log10(simps(x=wav_cm, y=wav_cm*S*thru_z(w))/(const*simps(x=w,y=w*thru_z(w)))) - 48.60

  return u,g,r,i,z

The nex figure looks at what happens to galaxy magnitudes and colors as the redshift increases.

In [ ]:
### FIGURE 8
figure(8,figsize=(8,8))

ages=[7.5,7.75,8,8.25,8.5,8.75,9,9.25,9.5,9.75,10]

alpha=-2.35

z=0.0
for age in ages:
  i=MIST.age_index(age)
  iso=MIST.isos[i]
  mags = ugriz_mags(iso,alpha,z=z)
  scatter(mags[1],mags[1]-mags[3],color='DodgerBlue')

z=1.0
for age in ages:
  i=MIST.age_index(age)
  iso=MIST.isos[i]
  mags = ugriz_mags(iso,alpha,z=z)
  scatter(mags[1],mags[1]-mags[3],color='LimeGreen')


z=2.0
for age in ages:
  i=MIST.age_index(age)
  iso=MIST.isos[i]
  mags = ugriz_mags(iso,alpha,z=z)
  scatter(mags[1],mags[1]-mags[3],color='Tomato')


xticks(fontsize=fs)
yticks(fontsize=fs)
xlabel(r'$M_g$',fontsize=fs)
ylabel(r'$g-i$',fontsize=fs)
title('Simple Stellar Population CMD',fontsize=fs);

Keep in mind when looking at this that our SSP is normalized to have an initial mass of 1$M_{\odot}$, so much less massive (and therefore less luminous) than a typical galaxy!  Ages shown in the figure go from log(age)=7.5 up to 10 in finer steps ($\Delta$log(age)=0.25) than we have used in previous figure.

Take a few minutes to ponder the effects.  The blue points have redshift z=0, the green points have redshift z=1, and the red points have redshift z=2.  How can we qualitatively explain what's going on here?


In Figure 9 we'll plot the absolute magnitude of a 1 Gyr SSP in the five filters as a function of redshift.

In [ ]:
### FIGURE 9
figure(9,figsize=(8,8))

alpha=-2.35
i = MIST.age_index(9) # 1 Gyr
iso = MIST.isos[i]

redshift = linspace(0,2,21)

for z in redshift:
  mags = ugriz_mags(iso,alpha,z=z)
  scatter(z,mags[0],color='DodgerBlue')
  scatter(z,mags[1],color='LimeGreen')
  scatter(z,mags[2],color='Gold')
  scatter(z,mags[3],color='DarkOrange')
  scatter(z,mags[4],color='FireBrick')

text(0,2.0,'u',color='DodgerBlue',fontsize=fs)
text(0,1.7,'g',color='LimeGreen',fontsize=fs)
text(0,1.4,'r',color='Gold',fontsize=fs)
text(0,1.1,'i',color='DarkOrange',fontsize=fs)
text(0,0.8,'z',color='FireBrick',fontsize=fs)

xticks(fontsize=fs)
yticks(fontsize=fs)
xlabel(r'Redshift $z$',fontsize=fs)
ylabel(r'Absolute Magnitude',fontsize=fs);

Notice that the effect of redshift on the SED of this SSP is different for each of the SDSS filters.  The u filter is monotonic, it gets fainter as z increases and likewise for g.  However, the r, i, and z filters have a different behavior: there is a redshift z > 0 for which the SSP reaches its maximum brightness.  Why?

## K Correction
The above gives rise to the "cosmological K correction".  It is a term in the expanded, cosmological version of the astronomical distance formula (i.e., when redshift is non-negligible) that accounts for the change in flux through a photometric filter:

$m_X-M_X = 5 \log_{10}D_L - 5 + K_{corr,X}$

Where $X$ here refers to a filter, such as u, g, r, i, or z.  The K correction is defined as the ratio of the redshifted flux through the filter to the z=0 case:

$K_{corr} = -2.5 \log_{10}\left( (1+z)\times\frac{\int SED_{z \neq 0}(\lambda) T(\lambda) \lambda d\lambda}{\int SED_{z=0}(\lambda) T(\lambda) \lambda d\lambda} \right)$

It should always be zero at z=0 because $\log_{10}(1)=0$.

One further, important point about the K correction is that it is quite sensitive to the SED of the object in question.  This applies not just to integrated spectrum of an SSP that we're considering here but also the, e.g., the observation of a Supernova at $z=1$ or 2.  One implication is that we have to have a pretty good idea in advance of what the objects spectrum looks like to be able to correctly apply the K correction.

In [ ]:
def ugriz_kcor(iso, alpha, z=0.0):
  M,N,w,S=SPS(iso=iso,alpha=alpha,z=z)
  M0,N0,w0,S0=SPS(iso=iso,alpha=alpha,z=0)
  wav_cm = 1e-8*w
  wav_cm0 = 1e-8*w0

  ku = -2.5*log10( (1+z) * simps(x=wav_cm, y=wav_cm*S*thru_u(w)) / simps(x=wav_cm0, y=wav_cm0*S0*thru_u(w0)) )
  kg = -2.5*log10( (1+z) * simps(x=wav_cm, y=wav_cm*S*thru_g(w)) / simps(x=wav_cm0, y=wav_cm0*S0*thru_g(w0)) )
  kr = -2.5*log10( (1+z) * simps(x=wav_cm, y=wav_cm*S*thru_r(w)) / simps(x=wav_cm0, y=wav_cm0*S0*thru_r(w0)) )
  ki = -2.5*log10( (1+z) * simps(x=wav_cm, y=wav_cm*S*thru_i(w)) / simps(x=wav_cm0, y=wav_cm0*S0*thru_i(w0)) )
  kz = -2.5*log10( (1+z) * simps(x=wav_cm, y=wav_cm*S*thru_z(w)) / simps(x=wav_cm0, y=wav_cm0*S0*thru_z(w0)) )

  return ku,kg,kr,ki,kz

And we can plot the k correction similarly to the previous plot.  It may take 1-2 minutes to make this plot.

In [ ]:
### FIGURE 10
figure(10,figsize=(8,8))

alpha=-2.35
i = MIST.age_index(9) # 1 Gyr
iso = MIST.isos[i]

redshift = linspace(0,2,21)

for z in redshift:
  kcors = ugriz_kcor(iso,alpha,z=z)
  scatter(z,kcors[0],color='DodgerBlue')
  scatter(z,kcors[1],color='LimeGreen')
  scatter(z,kcors[2],color='Gold')
  scatter(z,kcors[3],color='DarkOrange')
  scatter(z,kcors[4],color='FireBrick')

#text(0,3.5,'u',color='DodgerBlue',fontsize=fs)
#text(0,3.0,'g',color='LimeGreen',fontsize=fs)
#text(0,2.5,'r',color='Gold',fontsize=fs)
#text(0,2.0,'i',color='DarkOrange',fontsize=fs)
#text(0,1.5,'z',color='FireBrick',fontsize=fs)
grid()
ylim(-2,1)
xticks(fontsize=fs)
yticks(fontsize=fs)
xlabel(r'Redshift $z$',fontsize=fs)
ylabel(r'K Correction',fontsize=fs);

In order to see the effect of changing the spectrum on the K correction, try a different age.  You can try both a larger and a smaller age.





### THE END